# Import libs 

In [8]:
import pandas as pd
import hashlib
from dotenv import load_dotenv
import os

load_dotenv()
SALT = os.getenv("CUSTOMER_HASH_SALT")

# Read data


In [9]:
raw_data = pd.read_parquet("/home/huy/HCMUS/subject/Python_For_DS/data/raw/transaction_data.parquet")

In [10]:
raw_data.columns

Index(['datetime', 'ol_invoice_code', 'customer_code',
       'product_variant_barcode', 'quantity_order', 'selling_price'],
      dtype='object')

# Preprocess data


In [33]:
def preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    # Drop na values
    df = df.dropna(subset=['customer_code', 'ol_invoice_code'])

    # Normalizing data
    df['normalized_customer_code'] = data['customer_code'].str.lower()
    df['normalized_invoice_code'] = data['ol_invoice_code'].str.lower()
    return df


preprocessed_data = preprocess_data(raw_data)


# Hash 1 chiều (One-way Hash)

1. Hash 1 chiều là gì


<img src="/home/huy/HCMUS/subject/Python_For_DS/images/hash.jpg" width="600" height="auto" alt="Minh họa Hash Function"/>

**Hash 1 chiều** là một phép biến đổi toán học không đảo được được thiết ể chuyển dữ liệu gốc (input) thành một chuỗi đại diện (hash) sao cho:
- Không thể suy ra dữ liệu gốc từ hash
- Hash chỉ dùng để đại diện, không dùng để lưu trữ hay phục hồi thông tin gốc

2. Vì sao sử dụng hash ?


In [34]:

def hash_customer(code: str) -> str:
    combined = f"{code}{SALT}"
    return hashlib.sha256(combined.encode("utf-8")).hexdigest()

preprocessed_data['hashed_customer_code'] = preprocessed_data['normalized_code'].apply(hash_customer)
preprocessed_data['hashed_invoice_code'] = preprocessed_data['normalized_invoice_code'].apply(hash_customer)

In [35]:
preprocessed_data.columns


Index(['datetime', 'ol_invoice_code', 'customer_code',
       'product_variant_barcode', 'quantity_order', 'selling_price',
       'normalized_code', 'hashed_code', 'normalized_customer_code',
       'normalized_invoice_code', 'hashed_customer_code',
       'hashed_invoice_code'],
      dtype='object')

# Kiểm tra tính toàn vẹn của dữ liệu sau khi xử lý 

In [36]:
# Ensure 1–1 mapping: mỗi normalized_code phải sinh ra đúng 1 hashed_code (không collision, không gộp sai)
assert preprocessed_data['hashed_customer_code'].nunique() == preprocessed_data['normalized_customer_code'].nunique()
assert preprocessed_data['hashed_invoice_code'].nunique() == preprocessed_data['normalized_invoice_code'].nunique()

In [37]:
def post_process_data(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df.drop(columns=['customer_code',"normalized_code","ol_invoice_code", "normalized_invoice_code"], inplace=True)

    df.rename(columns={'hashed_code': 'customer_code',
                       'hashed_invoice_code': 'ol_invoice_code'}, inplace=True)
    return df

post_processed_data = post_process_data(preprocessed_data)

In [38]:
post_processed_data.to_parquet("/home/huy/HCMUS/subject/Python_For_DS/data/preprocessed/transaction_data_processed.parquet")